In [0]:
# On définit la base du chemin vers ton volume
# Si tu n'as pas créé de schéma spécifique, c'est 'default'
base_path = "/Volumes/workspace/default/bigdataproject/"


# On liste les fichiers que tu as uploadés
# Vérifie bien que les noms correspondent exactement à ce que tu as dans ton volume
path_names = f"{base_path}name.basics.tsv"
path_titles = f"{base_path}title.basics.tsv"
path_ratings = f"{base_path}title.ratings.tsv"

In [0]:
# Lecture du fichier TSV compressé
df_names = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("nullValue", "\\N") \
    .load(path_names)

# On compte le nombre de lignes
total_people = df_names.count()

print(f"Nombre total de personnes dans le dataset : {total_people}")